# Word Cloud - Tweeter

In [5]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as functions

spark = SparkSession.builder\

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/franciscofoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
